In [1]:
import numpy as np

In [2]:
def hms2dec(hours, minutes, seconds):
    return 15*(hours + minutes/60 + seconds/(60**2))

In [3]:
def dms2dec(degrees, minutes, seconds):
    sign = 1 if degrees > 0 else -1
    return sign*(abs(degrees) + minutes/60 + seconds/(60**2))

In [4]:
def import_bss():
    result = list()
    cat = np.loadtxt('bss.dat', usecols=range(1,7))
    for i in range(0, len(cat)):
        ra = hms2dec(cat[i,0], cat[i,1], cat[i,2])
        dec = dms2dec(cat[i,3], cat[i,4], cat[i,5])
        result.append((i+1, ra, dec))
    return result

In [5]:
def import_super():
    result = list()
    cat = np.loadtxt('super.csv', delimiter=',', skiprows=1, usecols=[0,1])
    for i in range(0, len(cat)):
        ra = cat[i,0]
        dec = cat[i,1]
        result.append((i+1, ra, dec))
    return result

In [6]:
def angular_dist(r1_deg, d1_deg, r2_deg, d2_deg):
    r1 = np.radians(r1_deg)
    d1 = np.radians(d1_deg)
    r2 = np.radians(r2_deg)
    d2 = np.radians(d2_deg)
    
    a = np.sin(np.abs(d1 - d2)/2)**2
    b = np.cos(d1) * np.cos(d2) * np.sin(np.abs(r1 - r2)/2)**2
    
    d = 2 * np.arcsin(np.sqrt(a+b))
    
    return np.degrees(d)

In [11]:
def find_closest(catalog, ra, dec):
    closest = {}
    closest_dist = 360
    for element in catalog:
        dist = angular_dist(ra, dec, element[1], element[2])
        if dist < closest_dist:
            closest_dist = dist
            closest = element
    return (closest[0], closest_dist)

In [14]:
cat = import_bss()
find_closest(cat, 175.3, -32.5)

(156, 3.7670580226469053)

In [15]:
cat = import_bss()
find_closest(cat, 32.2, 40.7)
(26, 57.729135775621295)

(26, 57.729135775621295)

In [20]:
def crossmatch(cat1, cat2, max_dist):
    match = list()
    no_match = list()
    for e1 in cat1:
        e2_id, e2_dist = find_closest(cat2, e1[1], e1[2])
        if e2_dist < max_dist:
            match.append((e1[0], e2_id, e2_dist))
        else:
            no_match.append(e1[0])
    return (match, no_match)

In [21]:
bss_cat = import_bss()
super_cat = import_super()
max_dist = 40/3600
matches, no_matches = crossmatch(bss_cat, super_cat, max_dist)
print(matches[:3])
print(no_matches[:3])
print(len(no_matches))

[(1, 2, 0.00010988610938710059), (2, 4, 0.0007649845967242494), (3, 5, 0.00020863352870707666)]
[5, 6, 11]
9


In [22]:
bss_cat = import_bss()
super_cat = import_super()
max_dist = 5/3600
matches, no_matches = crossmatch(bss_cat, super_cat, max_dist)
print(matches[:3])
print(no_matches[:3])
print(len(no_matches))

[(1, 2, 0.00010988610938710059), (2, 4, 0.0007649845967242494), (3, 5, 0.00020863352870707666)]
[5, 6, 11]
40
